In [1]:
import pickle
import layers

2024-04-22 14:37:11.288944: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 14:37:11.289202: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 14:37:11.291634: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 14:37:11.322809: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 14:37:11.907364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
x = layers.Input(2)
x = layers.Dense(x, 0.1, 3)
x = layers.Dense(x, 0.1, 3)
model = x
# x([
#     [1,2]
# ])

In [3]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('model.pkl', 'rb') as f:
    model2 = pickle.load(f)

In [4]:
model2([
    [1,2]
])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.6619774, 0.6060494, 0.6316492]], dtype=float32)>